In [155]:
import os
import numpy as np
import pandas as pd

Latitude="45"

# writing the path of the tables
RLF_path="/content/drive/MyDrive/EETBS 2021-2022/RLF Tables"
BeamIrradiance_path=os.path.join(RLF_path,"BeamIrradiance.csv")
DiffuseIrradiance_path=os.path.join(RLF_path,"DiffuseIrradiance.csv")
FFs_path=os.path.join(RLF_path,"FFs.csv")
IAC_cl_path=os.path.join(RLF_path,"IAC_cl.csv")
SLF_path=os.path.join(RLF_path,"SLF.csv")
windows_path=os.path.join(RLF_path,"windows.csv")
SomeIntShading_path=os.path.join(RLF_path,"SomeIntShading.csv")


#reading the tables as a data frame

BeamIrradiance_DF=pd.read_csv(BeamIrradiance_path,sep=";",index_col=0,header=0)
DiffuseIrradiance_DF=pd.read_csv(DiffuseIrradiance_path,sep=";",index_col=0,header=0)
DiffuseIrradiance_DF.index.name="A"
FFs_DF=pd.read_csv(FFs_path,sep=";",index_col=0,header=0)
IAC_cl_DF=pd.read_csv(IAC_cl_path,sep=";",index_col=1,header=0)
SLF_DF=pd.read_csv(SLF_path,sep=";",index_col=0,header=0)
windows_DF=pd.read_csv(windows_path,sep=";",index_col=0,header=0)
SomeIntShading_DF=pd.read_csv(SomeIntShading_path,sep=";",index_col=0,header=0)




In [156]:
#deriving the SLF value

def SLF_finder(row):
  Direction=row["Direction"]
  SLF_Value=SLF_DF.loc[Direction,Latitude]
  return(SLF_Value)

windows_DF.loc[:,"SLF"]=windows_DF.apply(SLF_finder,axis=1)







In [157]:
#deriving_F_shading

def F_sh(row):
  SLF=row["SLF"]
  Doh=row["Doh"]
  Xoh=row["Xoh"]
  Height=row["Height"]
  Value_of_F=min(1,max(0,(SLF*Doh-Xoh)/Height))
  return(Value_of_F)

windows_DF.loc[:,"Fshd"]=windows_DF.apply(F_sh,axis=1)





In [158]:
#deriving_IAC_cl

def IAC_cl_finder(row):
  Window_ID=row["Window_ID"]
  IntShading_ID=row["IntShading_ID"]
  Value_of_IAC=IAC_cl_DF.loc[Window_ID,IntShading_ID]
  return(Value_of_IAC)

windows_DF.loc[:,"IAC_cl"]=windows_DF.apply(IAC_cl_finder,axis=1)
windows_DF.loc[:,"IAC"]=1+windows_DF.loc[:,"IntShading_closeness"]*(windows_DF.loc[:,"IAC_cl"]-1)

In [159]:
#deriving_Ed

def Ed_finder(row):
  Direction=row["Direction"]
  Value_of_Ed=DiffuseIrradiance_DF.loc[Direction,"45"]
  return(Value_of_Ed)
windows_DF.loc[:,"Ed"]=windows_DF.apply(Ed_finder,axis=1)




In [160]:
#deriving_ED

def ED_finder(row):
  Direction=row["Direction"]
  Value_of_ED=BeamIrradiance_DF.loc[Direction,"45"]
  return(Value_of_ED)
windows_DF.loc[:,"ED"]=windows_DF.apply(ED_finder,axis=1)




In [161]:
#deriving FFs

def FFs_finder(row):
  Direction=row["Direction"]
  Value_of_FFs=FFs_DF.loc[Direction,"SingleFamilyDetached"]
  return(Value_of_FFs)
windows_DF.loc[:,"FFs"]=windows_DF.apply(FFs_finder,axis=1)


In [162]:
#calculating PXI

windows_DF.loc[:,"PXI"]=windows_DF.loc[:,"Tx"]*(windows_DF.loc[:,"Ed"]+(1-windows_DF.loc[:,"Fshd"])*windows_DF.loc[:,"ED"])


windows_DF

,Direction,width,Height,Area,Window_ID,Frame_type,Frame_material,U,SHGC,HF,Q heating,SLF,Doh,Xoh,Fshd,IntShading_ID,IntShading_closeness,IAC_cl,IAC,Tx,Ed,ED,PXI,FFs,C_value,CF,Qcooling
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,
east,E,8,1.8,0,5c,Fixed,Wood,0,0,0,0,1.0,0.9,0.3,0.333333,DrapesLightOpen,0.4,0.73,0.892,1.00,188,559,560.666667,0.31,0,0,0
west,W,8,1.8,0,5c,Fixed,Wood,0,0,0,0,1.0,0.9,0.3,0.333333,DrapesLightOpen,0.4,0.73,0.892,1.00,188,559,560.666667,0.56,0,0,0
south-Fixed,S,2,1.8,0,5c,Fixed,Wood,0,0,0,0,2.7,0.9,0.3,1.000000,DrapesLightOpen,0.4,0.73,0.892,1.00,209,348,209.000000,0.47,0,0,0
south-Operable,S,2,1.8,0,5c,Operable,Wood,0,0,0,0,2.7,0.9,0.3,1.000000,DrapesLightOpen,0.4,0.73,0.892,0.64,209,348,133.760000,0.47,0,0,0
